<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2024-02-26 13:26:00--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.103.159, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2024-02-26 13:26:00--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.2’

SO_vectors_200.bin? 100%[===================>]   1.35G  28.8MB/s    in 51s     

2024-02-26 13:26:51 (27.3 MB/s) - ‘SO_vectors_200.bin?download=1.2’ saved [1453905423/1453905423]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
# method most_simmilar
if 'cat' in wv_embeddings and 'dog' in wv_embeddings:
    similar_words = wv_embeddings.most_similar('dog', topn=5)
    if 'cat' in dict(similar_words):
        cat_rank = [word for word, score in similar_words].index('cat') + 1
        print(f"'cat' входит в топ 5 ближайших к 'dog' и занимает {cat_rank}-е место.")
    else:
        print("'cat' не входит в топ 5 ближайших к 'dog'.")
else:
    print("Одно из слов 'cat' или 'dog' отсутствует в словаре.")

'cat' не входит в топ 5 ближайших к 'dog'.


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        return: векторное представление для вопроса """

    que2vec = np.zeros(dim, dtype='float32')
    lenght = 0
    for word in tokenizer.tokenize(question.lower()):
      if word in embeddings:
        que2vec += embeddings[word]
        lenght += 1

    return que2vec / lenght if lenght > 0 else que2vec

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
que = 'I love neural networks'
round(question_to_vec(que, wv_embeddings, tokenizer)[2],2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

ответ: max Hits@47 равен единице,это будет тогда, когда все дубликаты в вопросах расположены среди первых 47 вариантов ответов. min DCG@1 равен нулю, это будет тогдв, когда дубликаты не расположены на первых местах, слеовательно, максимум выражения равен единице

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
rank_q_i_prime = 9
K = 10
dcg_10 = round(1 / np.log2(1+rank_q_i_prime),1)
print(f"DCG@10: {dcg_10}")

DCG@10: 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = round(sum([np.zeros(1)[0] if rank > k else np.ones(1)[0] for rank in dup_ranks]) / len(dup_ranks),4)
    return hits_value

In [ ]:

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = round(sum([np.zeros(1)[0] if rank > k else np.log2(1 + rank)**(-1) for rank in dup_ranks]) / len(dup_ranks),4)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.6309, 0.6309, 0.6309]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
unzip:  cannot find or open content/drive/MyDrive/data_sets/stackoverflow_similar_questions.zip, content/drive/MyDrive/data_sets/stackoverflow_similar_questions.zip.zip or content/drive/MyDrive/data_sets/stackoverflow_similar_questions.zip.ZIP.


In [ ]:
!unzip /content/drive/MyDrive/data_sets/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/data_sets/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    que2vec = question_to_vec(question, embeddings, tokenizer)
    result = []
    for idx, elem in enumerate(candidates):
        candidate2vec = question_to_vec(elem, embeddings, tokenizer)
        result.append((idx, elem, cosine_similarity(que2vec.reshape(1, -1), candidate2vec.reshape(1, -1))[0][0]))

    result.sort(key=lambda x: x[2], reverse=True)
    return [elem[:-1] for elem in deepcopy(result)]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ: получено 021**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
my_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def preproc_nltk(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in my_stopwords])

In [ ]:
pairs = [pair[0] + str(' ') + pair[1] for pair in train_data]
words = [preproc_nltk(question).split() for question in pairs]

In [ ]:
from gensim.models import Word2Vec
# Skip-Gram

for window in range(4,10,2):
    embeddings_trained = Word2Vec(words,
                                vector_size=200,
                                min_count=5,
                                window=window,
                                workers=4,
                                sg=1).wv
    wv_ranking = []
    max_validation_examples = 1000
    tokenizer = MyTokenizer()
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    print(f'window = {window}')
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

window = 4


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.503 | Hits@   1: 0.503
DCG@   5: 0.587 | Hits@   5: 0.661
DCG@  10: 0.603 | Hits@  10: 0.712
DCG@ 100: 0.644 | Hits@ 100: 0.901
DCG@ 500: 0.654 | Hits@ 500: 0.982
DCG@1000: 0.656 | Hits@1000: 1.000


  0%|          | 0/3760 [00:00<?, ?it/s]

...

DCG@   1: 0.515 | Hits@   1: 0.515
DCG@   5: 0.599 | Hits@   5: 0.671
DCG@  10: 0.620 | Hits@  10: 0.737
DCG@ 100: 0.657 | Hits@ 100: 0.913
DCG@ 500: 0.666 | Hits@ 500: 0.984
DCG@1000: 0.668 | Hits@1000: 1.000


  0%|          | 0/3760 [00:00<?, ?it/s]

window = 8


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.522 | Hits@   1: 0.522
DCG@   5: 0.606 | Hits@   5: 0.677
DCG@  10: 0.625 | Hits@  10: 0.737
DCG@ 100: 0.663 | Hits@ 100: 0.916
DCG@ 500: 0.672 | Hits@ 500: 0.986
DCG@1000: 0.674 | Hits@1000: 1.000


In [ ]:
from gensim.models import Word2Vec
# CBOW

for window in range(4,10,2):
    embeddings_trained = Word2Vec(words,
                                vector_size=200,
                                min_count=5,
                                window=window,
                                workers=4,
                                sg=0).wv
    wv_ranking = []
    max_validation_examples = 1000
    tokenizer = MyTokenizer()
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    print(f'window = {window}')
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

window = 4


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.395 | Hits@   1: 0.395
DCG@   5: 0.490 | Hits@   5: 0.574
DCG@  10: 0.513 | Hits@  10: 0.646
DCG@ 100: 0.559 | Hits@ 100: 0.872
DCG@ 500: 0.573 | Hits@ 500: 0.977
DCG@1000: 0.575 | Hits@1000: 1.000


  0%|          | 0/3760 [00:00<?, ?it/s]

...

DCG@   1: 0.401 | Hits@   1: 0.401
DCG@   5: 0.496 | Hits@   5: 0.578
DCG@  10: 0.523 | Hits@  10: 0.662
DCG@ 100: 0.568 | Hits@ 100: 0.883
DCG@ 500: 0.581 | Hits@ 500: 0.977
DCG@1000: 0.583 | Hits@1000: 1.000


  0%|          | 0/3760 [00:00<?, ?it/s]

window = 8


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.505 | Hits@   5: 0.588
DCG@  10: 0.527 | Hits@  10: 0.656
DCG@ 100: 0.574 | Hits@ 100: 0.884
DCG@ 500: 0.587 | Hits@ 500: 0.978
DCG@1000: 0.589 | Hits@1000: 1.000


In [ ]:
from nltk.tokenize import WordPunctTokenizer

# возьмем другой токенайзер и обучим word2vec Skip-Gram на оптимальном window = 8
embeddings_trained = Word2Vec(words,
                                vector_size=200,
                                min_count=5,
                                window=window,
                                workers=4,
                                sg=1).wv
wv_ranking = []
max_validation_examples = 1000
tokenizer = WordPunctTokenizer()
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.508 | Hits@   1: 0.508
DCG@   5: 0.591 | Hits@   5: 0.665
DCG@  10: 0.610 | Hits@  10: 0.726
DCG@ 100: 0.646 | Hits@ 100: 0.900
DCG@ 500: 0.657 | Hits@ 500: 0.980
DCG@1000: 0.659 | Hits@1000: 1.000


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

class SpacyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        results = nlp(text)
        return [token.lemma_ for token in results]

# нормализация слов
def preproc_nltk_norm(text):
    text = ' '.join([word for word in word_tokenize(text.lower()) if word not in my_stopwords])
    return SpacyTokenizer().tokenize(text)
words_norm = [preproc_nltk_norm(question) for question in pairs]

In [ ]:
# с нормализацией
embeddings_trained = Word2Vec(words_norm,
                                vector_size=200,
                                min_count=5,
                                window=8,
                                workers=4,
                                sg=1).wv
wv_ranking = []
max_validation_examples = 1000
tokenizer = MyTokenizer()
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.364 | Hits@   1: 0.364
DCG@   5: 0.437 | Hits@   5: 0.503
DCG@  10: 0.454 | Hits@  10: 0.555
DCG@ 100: 0.495 | Hits@ 100: 0.754
DCG@ 500: 0.519 | Hits@ 500: 0.941
DCG@1000: 0.525 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


- После нескольких эксперементов значение параметра window около  признано оптимальным. Вообще бОльшие значения window позволяют модели рассматривать слова в более широком контексте, что может быть полезным для захвата долгосрочных зависимостей. Однако это также может сделать обучение более ресурсоемким.
- В данной задаче нормализачия значительно ухудшила результат
- Skip-Gram (алгоритм принимает центральное слово и пытается предсказать окружающие слова) показал себя сильно лучше, чем CBOW (алгоритм принимает на вход окружающие слова (контекст) и пытается предсказать центральное слово). Но CBOW обучался в неколько раз дольше. Skip-Gram в свою очередь лучше справляется с редкими словами,и чуть лучше подходит для создания векторных представлений.
- Плохое качество решения задачи получилось, потому что имеется достаточно специфический корпус, а так же используется самый обычный word2vec
- Для лучшего решения следовало бы применить более современные технологии, например, можно было бы взять контекстуализированные эмбеддинги типа bert эмбеддингов, которые будут лучше понимать контекст слов, подобно Apple, которое в данном корпусе скорее означает компанию, чем яблоко